<a href="https://colab.research.google.com/github/lavasany/DeepLearning/blob/main/3_Reimplementing_layers_%DA%AF%D8%B1%D9%81%D8%AA%D9%87_%D8%B4%D8%AF%D9%87_%D8%A7%D8%B2%D9%81%D8%B5%D9%84_%D8%AF%D9%88%D9%85_%DA%A9%D8%AA%D8%A7%D8%A8_%DB%8C%D8%A7%D8%AF_%DA%AF%DB%8C%D8%B1%DB%8C_%D8%B9%D9%85%DB%8C%D9%82_%D8%A8%D8%A7_%D9%BE%D8%A7%DB%8C%D8%AA%D9%88%D9%86_%DA%86%D9%88%D9%84%D8%AA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
import numpy as np

**Preparing the image data**


In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

11490434/11490434 [==============================] - 0s 0us/step


**__call__ in Python**

In [3]:
class A:
    def __init__(self, a, b):
        self.a=a
        self.b=b
    def __call__(self, x):
        return self.a*x+self.b
    @property
    def get_a(Self):
        return Self.a

In [4]:
a = A(10, 5)


In [5]:
a.get_a

10

In [6]:
a(2)

25

**A simple Dense class**

In [7]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation

        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):
        return [self.W, self.b]

**A simple Sequential class**

In [8]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
           x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

**The network architecture**

In [9]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

**Model weights!**

In [10]:
model(train_images[0:32])
weights_and_bias = model.weights
for item in weights_and_bias:
    print(item.name, item.shape)

Variable:0 (784, 512)
Variable:0 (512,)
Variable:0 (512, 10)
Variable:0 (10,)


**Running one training step**

In [11]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [12]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)

In [13]:
from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

**A batch generator**

In [14]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

**The full training loop**

In [15]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")

**fit the model with custom train loop!**

In [16]:
fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 3.80
loss at batch 100: 2.26
loss at batch 200: 2.21
loss at batch 300: 2.09
loss at batch 400: 2.27
Epoch 1
loss at batch 0: 1.92
loss at batch 100: 1.90
loss at batch 200: 1.84
loss at batch 300: 1.71
loss at batch 400: 1.88
Epoch 2
loss at batch 0: 1.60
loss at batch 100: 1.60
loss at batch 200: 1.52
loss at batch 300: 1.42
loss at batch 400: 1.56
Epoch 3
loss at batch 0: 1.34
loss at batch 100: 1.36
loss at batch 200: 1.25
loss at batch 300: 1.20
loss at batch 400: 1.31
Epoch 4
loss at batch 0: 1.14
loss at batch 100: 1.17
loss at batch 200: 1.05
loss at batch 300: 1.04
loss at batch 400: 1.13
Epoch 5
loss at batch 0: 0.99
loss at batch 100: 1.03
loss at batch 200: 0.91
loss at batch 300: 0.92
loss at batch 400: 1.01
Epoch 6
loss at batch 0: 0.88
loss at batch 100: 0.92
loss at batch 200: 0.81
loss at batch 300: 0.83
loss at batch 400: 0.92
Epoch 7
loss at batch 0: 0.80
loss at batch 100: 0.84
loss at batch 200: 0.73
loss at batch 300: 0.76
loss at batch 40

**Evaluating the model on new data**

In [17]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.81
